In [420]:
import pandas as pd
import numpy as np
from scipy.signal import iirnotch, freqz, lfilter, butter
from scipy.stats import skew, kurtosis, zscore

In [1440]:
# Cargar el archivo CSV proporcionado
file = 'C:/Users/HP/OneDrive/Escritorio/TFG/Psychopy/Yoli_2musica_electronica.csv'
df = pd.read_csv(file)

# Eliminar los datos del Channel 5 (última columna)
df_filtered = df.iloc[:, :-1] #Fichero sin el último canal


In [1441]:
fs = 256 #Frecuencia de muestreo
f0 = 60 #Frecuencia a eliminar
Q = 30

#Filtro notch
b, a = iirnotch(f0, Q, fs)

In [1442]:
#Filtro paso banda --> 4-45 Hz para eliminar estas bandas de frecuencia
# Función para crear un filtro pasa-banda
def butter_bandpass(baja_frec, alta_frec, fs, order=4):
    nyq = 0.5 * fs
    baja = baja_frec / nyq
    alta = alta_frec / nyq
    b, a = butter(order, [baja, alta], btype='band')
    return b, a
# Función para aplicar un filtro pasa-banda
def bandpass_filter(data, baja_frec, alta_frec, fs, order=4):
    b, a = butter_bandpass(baja_frec, alta_frec, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [1443]:
#Aplicacion de los dos filtros
df_data = df_filtered.copy()
for channel in df_data.columns[1:]:  # Evitar la columna 'Timestamp'
    df_data[channel] = lfilter(b, a, df_data[channel].values)

In [1444]:
df_data_filter = df_data.copy()

# Aplicar filtro pasa-banda 4-45 Hz
baja_frec = 4
alta_frec = 45

#Aplico filtro paso banda para quedarme con la banda de 4-45 Hz
for channel in df_data_filter.columns[1:]:  # Evitar la columna 'Timestamp'
    df_data_filter[channel] = bandpass_filter(df_data_filter[channel].values, baja_frec, alta_frec,fs)


In [1445]:
delta_t = 1 / fs  # Intervalo de tiempo entre puntos consecutivos
num_bins = 8  # Número de bins para el histograma

# Función para calcular los coeficientes fa y fb para cada punto en un frame
def calculate_fa_fb_for_frame(frame, delta_t,n):
    fa = []
    fb = []
    inicio = n
    for i in range(1, len(frame)):  # Recorrer todos los puntos del frame excepto el primero
        # Calcular fa: |xn - x(n-1)| / delta_t
        if n > inicio + 1:
            fa.append(np.abs(frame[n] - frame[n-1]) / delta_t)
        # Calcular fb: |xn - x(n-2)| / delta_t
        if n > inicio + 2:
            fb.append(np.abs(frame[n] - frame[n-2]) / delta_t)
        n= n+1
    return fa, fb


In [1446]:
def calculate_frequency_histogram(frame, fs, num_bins):
    fft_result = np.fft.fft(frame)  # FFT de la señal
    fft_freqs = np.fft.fftfreq(len(frame), 1/fs)  # Frecuencias correspondientes
    fft_magnitudes = np.abs(fft_result)  # Magnitud de las componentes frecuenciales
    
    # Calcular histograma en el dominio de la frecuencia (frecuencias positivas)
    freq_hist, _ = np.histogram(fft_magnitudes[:len(fft_magnitudes)//2], bins=num_bins, density=True)
    
    return freq_hist

In [1447]:
# Parámetros de frecuencia para las diferentes bandas
num_frames = 21
overlap_ratio = 0.75

feature_vectors = []

# Dividir el DataFrame en 21 frames con solapamiento del 75%
total_samples = len(df_data_filter)
frame_size = total_samples / (num_frames + (num_frames - 1) * overlap_ratio) #tamaño de cada frame
frame_size = int(frame_size)
overlap = int(frame_size * overlap_ratio)

for i in range(num_frames):
    start_index = i * (frame_size - overlap)
    end_index = start_index + frame_size
    
    # Crear el vector para almacenar las características
    feature_vector = []
    for channel in df_data_filter.columns[1:]:
            frame = df_data_filter[channel].iloc[start_index:end_index].copy()
            # Calcular características
            #feature_vector.append(frame.mean())         # Media
            #feature_vector.append(np.median(frame))     # Mediana
            #feature_vector.append(frame.std())          # Desviación estándar
            #feature_vector.append(np.mean(zscore(frame)))  # Z-score
            #feature_vector.append(skew(frame))          # Skewness (Asimetría)
            #feature_vector.append(kurtosis(frame))      # Kurtosis

            fa, fb = calculate_fa_fb_for_frame(frame, delta_t,start_index)
        
            if len(frame) > 0:  # Verificar si el frame no está vacío y tiene variabilidad
                hist_Xn = calculate_frequency_histogram(frame, fs, num_bins)
                feature_vector.extend(hist_Xn)
            
            # Histograma de los coeficientes fa (evitar NaN e Inf)
        
            if len(fa) > 0:
                hist_fa = calculate_frequency_histogram(fa, fs, num_bins)
                feature_vector.extend(hist_fa)
            
            # Histograma de los coeficientes fb (evitar NaN e Inf)
            if len(fb) > 0:
                hist_fb = calculate_frequency_histogram(fb, fs, num_bins)
                feature_vector.extend(hist_fb)
                 #Añadir el vector de características a la lista
    feature_vectors.append(feature_vector)

# Convertir la lista de vectores de características en un DataFrame para su análisis
df_features = pd.DataFrame(feature_vectors, columns=[f'feature_{i+1}' for i in range(96)])

# Guardar el DataFrame de características en un archivo CSV si es necesario
df_features.to_csv('C:/Users/HP/OneDrive/Escritorio/TFG/Preprocesado/feature_Yoli_2_electronicaDHMS.csv', index=False)

print("Proceso completado y las características se han guardado en 'feature_vectors.csv'")

Proceso completado y las características se han guardado en 'feature_vectors.csv'
